<h1>Textbook RSA</h1>

In [ ]:
from Crypto.Util.number import *
from Crypto import Random
import Crypto
import binascii
from xgcd import * 
from print_bytes_util import print_bytes

Task: Try different bit length and messages.

In [ ]:
rsa_bit_length = 1024
plain_text = 'Hello World'

<h2>RSA Key Generation</h2>

In [ ]:
#Generate two large random primes, p and q, of approximately equal size 
p = Crypto.Util.number.getPrime(rsa_bit_length//2, randfunc=Crypto.Random.get_random_bytes)
q = Crypto.Util.number.getPrime(rsa_bit_length//2, randfunc=Crypto.Random.get_random_bytes)

#Calculate the modulus n:  n = p ∙ q
n = p*q

#Calculate 𝛗(𝒏)= (p-1) ∙(q-1)
PHI=(p-1)*(q-1)

#Choose public exponent e co prime to 𝛗(𝒏) and 𝐞≠±𝟏
e=65537

#Calculate secret exponent 𝒅=𝒆^−𝟏 (𝒎𝒐𝒅𝛗𝒏)
d= xgdc(e, PHI)

print(f'RSA key length = {rsa_bit_length} bits')
print(f'Message (plain text) = {plain_text}\n')

print('prime number p = 0x' + str(hex(p))[2:].upper() )
print('prime number q = 0x' + str(hex(q))[2:].upper() + '\n')

print('Public Key (e, n):')
print('  e = 0x' + str(hex(e))[2:].upper() )
print('  n = 0x' + str(hex(n))[2:].upper() + '\n')

print('Private Key (d, n):')
print('  d = 0x' + str(hex(d))[2:].upper() )
print('  n = 0x' + str(hex(n))[2:].upper() + '\n')


<h2>RSA Sign - Alice generates a RSA signature</h2>
Signing is using the private key {d, n}<br>
* Calculate the message hash: h = hash(M)<br>
* Sign h to calculate the signature: s = h^d (mod n)


In [ ]:
from cryptography.hazmat.primitives import hashes

print('Message = ' + plain_text)
digest = hashes.Hash( hashes.SHA1() )
alice_baMessage = plain_text.encode('utf-8')
digest.update(alice_baMessage)
alice_hashFromMessage_bytes = digest.finalize()

print('Alice: h = hash(message) 0x' + alice_hashFromMessage_bytes.hex(' ').upper())

alice_hashFromMessage_long =  bytes_to_long(alice_hashFromMessage_bytes)
alice_rsa_signature = pow(alice_hashFromMessage_long, d, n)

print_bytes('Alice: RSA Signature s = 0x', long_to_bytes(alice_rsa_signature), 16)


<h2>RSA Verify- Bob performs the signature verification</h2>
Verify is using the public key {d, n}<br>
* Calculate the message hash: h = hash(M)<br>
* Verify the signature: h' = s^e (mod n)<br>
* Compare h with h' to check if the signature is valid or not

In [ ]:
from cryptography.hazmat.primitives import hashes

#plain_text = 'Mallory has modified the message'

print('Message = ' + plain_text)
digest = hashes.Hash( hashes.SHA1() )
baMessage = plain_text.encode('utf-8')
digest.update(baMessage)
bob_hashFromMessage_bytes = digest.finalize()
print_bytes('Bob: h =  hash(message) 0x', bob_hashFromMessage_bytes, 16)

bob_hashFromSignature_long  = pow(alice_rsa_signature, e, n)
bob_hashFromSignature_bytes = long_to_bytes(bob_hashFromSignature_long)

print_bytes('Bob: h = s^e (mod n) = 0x', bob_hashFromSignature_bytes, 16)

print("Signature valid:", bob_hashFromMessage_bytes == bob_hashFromSignature_bytes)